In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?"
params = "ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Kama Ayurveda",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda
Scraping Page 2: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?page_no=2&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop
Scraping Page 3: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?page_no=3&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop
Scraping Page 4: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?page_no=4&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop
Scraping Page 5: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?page_no=5&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop
Scraping Page 6: https://www.nykaa.com/luxe/brands/kama-ayurveda/c/2421?page_no=6&sort=popularity&ptype=brand&id=2421&root=brand_menu,top_brands,Kama%20Ayurveda&eq=desktop
Scrap

In [3]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Kama Ayurveda', 'Product Name': 'Kama Ayurveda Pure Rosewater Duo(2 pcs)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '29251ratings&4976reviews', 'Product Original Price': '₹3190', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Kama Ayurveda', 'Product Name': 'Kama Ayurveda Pure Rose Water & Kumkumadi Serum(2 pcs)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '32608ratings&5051reviews', 'Product Original Price': '₹2690', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Kama Ayurveda', 'Product Name': 'Kama Ayurveda Hair Care Essentials(2 pcs)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '5268ratings&881reviews', 'Product Original Price': '₹2790', 'Product Offer Price': 'N/A', 'Product Discount': 'No

In [4]:
kamaAyurveda = pd.DataFrame(products)
kamaAyurveda

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Kama Ayurveda,Kama Ayurveda Pure Rosewater Duo(2 pcs),4.5/5,29251ratings&4976reviews,₹3190,N/A,No Discount,BESTSELLER,In Stock
1,Kama Ayurveda,Kama Ayurveda Pure Rose Water & Kumkumadi Seru...,4.5/5,32608ratings&5051reviews,₹2690,N/A,No Discount,None,In Stock
2,Kama Ayurveda,Kama Ayurveda Hair Care Essentials(2 pcs),4.4/5,5268ratings&881reviews,₹2790,N/A,No Discount,None,In Stock
3,Kama Ayurveda,Kama Ayurveda Organic Henna & Indigo Powder(2 ...,4.4/5,2204ratings&224reviews,₹1990,N/A,No Discount,None,In Stock
4,Kama Ayurveda,Kama Ayurveda Pure Rose Water Toner- Tightens ...,4.5/5,29251ratings&4976reviews,₹1595,N/A,No Discount,BESTSELLER,In Stock
...,...,...,...,...,...,...,...,...,...
113,Kama Ayurveda,Kama Ayurveda Ultra Nourishers Travel Kit(75 g),4.5/5,39ratings,₹1450,N/A,No Discount,None,Out Of Stock
114,Kama Ayurveda,Kama Ayurveda Glow All Day Travel Kit(55ml+21g),4.3/5,60ratings&7reviews,₹1675,N/A,No Discount,None,Out Of Stock
115,Kama Ayurveda,Kama Ayurveda One Year Before The Wedding (New...,4.3/5,19ratings&1reviews,₹1800,N/A,No Discount,None,Out Of Stock
116,Kama Ayurveda,Kama Ayurveda The Ultimate Wedding Regime(545ml),N/A,None,₹6535,N/A,No Discount,None,Out Of Stock


In [6]:
# kamaAyurveda.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Kama Ayurveda Products_Nykaa.csv", index = False)